# CT Spyder Field
## A python package to describe spiders' visual fields using CT scan data only

This jupyter notebook will guide you through the use the software from data (images coming from a segmented CT) to fields (plots and raw data export)

For a guide on how to install the software, the requirements and how to prepare CT scans, go to README.md

### Load packages
As a first step, let's load the library and other needed packages

In [6]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../')) # or the path to your source code
sys.path.insert(0, module_path)
from ctSpyderFields import ctSpyderFields
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Definitions
Now, let's provide some definitions. These are specific to your machine and data

In [7]:
path = '/path/to/foder/where/image/data/is/located/' #remember the final "/"
paramspath = 'path/to/where/parameters/for/images/is/located/params.yaml'

params.yaml contains the color definition for importing points from image stack from AMIRA/DRAGONFLY. This is needed if the image stack presents different ROIs with different colors. Provide in the yaml the RGB value for each ROIs, giving a minimum and a maximum value (images may not be precise and especially around the edges may blur slightly with the background). An example params.yaml is provided for the structure

In [8]:
labelnames = {'AME': {'Lens': 'Lens_AME', 'Retina': 'Retina_AME'},
                  'ALE': {'Lens': 'Lens_ALE', 'Retina': 'Retina_ALE'},
                  'PME': {'Lens': 'Lens_PME', 'Retina': 'Retina_PME'},
                  'PLE': {'Lens': 'Lens_PLE', 'Retina': 'Retina_PLE'},
                  'Markers': {'center': 'Marker_center',
                              'front': 'Marker_front', 'back': 'Marker_back',
                              'bottom': 'Marker_bottom', 'top': 'Marker_top',
                              'left': 'Marker_left', 'right': 'Marker_right'}}

labelnames dictionary provides the name of the image stack associated with each ROI. Change them accordingly to your image naming (e.g., instead of 'Lens_AME' write 'Image_name_without_incremental_number_nor_file_type'). If the same image stack contains more than one ROI in different colors, provide the same text (e.g. {'AME': {'Lens': 'Full_AME_image_name', 'Retina': 'Full_AME_image_name'})

### Import image stacks
First, we create the object 

In [ ]:
MySpiderObjectName = ctSpyderFields.Spider(workdir=path, label_names=labelnames, voxelsize=0.001, paramspath=paramspath) 
# remember to set voxelsize as given by your CT analysis software

Then, we load the images.

If you are using an image stack where ROIs have different colors, or in general the images are RGB, use

In [ ]:
MySpiderObjectName.load_all_labels_split(style='color')
MySpiderObjectName.find_eyes_points(style='color')
MySpiderObjectName.compute_cephalothorax(style='color')

If you are using an image stack where ROIs are all in different stacks, are white and the baground is black (0s and 1s), use

In [ ]:
MySpiderObjectName.load_all_labels_split(style='binary')
MySpiderObjectName.find_eyes_points(style='binary')
MySpiderObjectName.compute_cephalothorax(style='binary')

### Save and Reload
Loading images is time consuming. If you want, you can at this stage save your data in pickle file. Next time you can start from the point data directly, using a lot less memory and taking less time

In [ ]:
GenusSpecies.save(filename='GenusSpecies')
#GenusSpecies.save('GenusSpecies', type='h5') # Alternatively, specify the file extension. the default is .pickle

from now on, reload the data with

In [ ]:
GenusSpecies = Ct.Spider(workdir=path, voxelsize=0.001, paramspath=paramspath) 
# REMEMBER! You always have to recreate the spider object for every new file.
GenusSpecies.load(filename='GenusSpecies', type='pickle')

Note that saving only save dot clouds. For the projection data you will have to compute it every time.

### Anatomy and orientation
Now, we need to go from the provided ROIs to anatomically meaningful solids. Specifically we need to define:
- The spider orientation relative to the CT scanner
- The eyes and retina surfaces
- The eyes focal point

Let's start with spider orientation

In [ ]:
GenusSpecies.head_SoR(flipX=False, flipZ=False, plot=False)

This code takes the 7 body markers and create a new set of axis. This is:
- centered on the center marker
- with its x axis aligned with the back-front like, with positive xs going towards front
- with its z axis perpendicular to x, parallel to the top-bottom axis with positive zs towards top
- with its y axis perpendicular to x and z

Note that this function has a plot argument. This is used to check for errors in this reorientation step. Try:

In [ ]:
GenusSpecies.head_SoR(flipX=False, flipZ=False, plot=True)

If from the plot you notice some errors, you can fix it with the other two arguments. For example, if the Z axis points towards bottom rather than top, do:

In [ ]:
GenusSpecies.head_SoR(flipX=False, flipZ=True, plot=True)

Remember. This code saves the roto-translational matrix for the spider every time you run it. Be sure that your last run had the correct orientation.